# VII - Parallel and Distributed Execution
In this notebook, we will execute training across multiple nodes (or in parallel across a single node over multiple GPUs). We will train an image classification model with Resnet20 on the CIFAR-10 data set across multiple nodes in this notebook.

Azure Batch and Batch Shipyard have the ability to perform "gang scheduling" or scheduling multiple nodes for a single task. This is most commonly used for Message Passing Interface (MPI) jobs.

* [Setup](#section1)
* [Configure and Submit MPI Job and Submit](#section2)
* [Delete Multi-Instance Job](#section3)

<a id='section1'></a>

## Setup

Create a simple alias for Batch Shipyard

In [ ]:
%alias shipyard SHIPYARD_CONFIGDIR=config python $HOME/batch-shipyard/shipyard.py %l

Check that everything is working

In [ ]:
shipyard

Read in the account information we saved earlier

In [ ]:
import json
import os

def read_json(filename):
    with open(filename, 'r') as infile:
        return json.load(infile)
    
def write_json_to_file(json_dict, filename):
    """ Simple function to write JSON dictionaries to files
    """
    with open(filename, 'w') as outfile:
        json.dump(json_dict, outfile)

account_info = read_json('account_information.json')

storage_account_key = account_info['storage_account_key']
storage_account_name = account_info['storage_account_name']
STORAGE_ALIAS = account_info['STORAGE_ALIAS']

We will need to delete the pool from earlier as we need a different Docker image on the pool. Due to limited core quota on a default Batch account, we'll need to wait for this pool to delete first before proceeding.

In [ ]:
shipyard pool del -y --wait

In [ ]:
IMAGE_NAME = 'alfpark/cntk:2.1-gpu-1bitsgd-py35-cuda8-cudnn6-refdata'

This CNTK image contains the 1-bit SGD version of CNTK for use with GPUs. Additionally it already comes preloaded with the CIFAR-10 reference data, so there is no need to download and convert the training data as it is already baked into the image.

Note that if we were using Infiniband/RDMA enabled instances, we would opt to use the `intelmpi` versions of the CNTK images instead.

Now we will create the config structure:

In [ ]:
config = {
    "batch_shipyard": {
        "storage_account_settings": STORAGE_ALIAS
    },
    "global_resources": {
        "docker_images": [
            IMAGE_NAME
        ]
    }
}

Now we'll create the pool specification with a few modifications for this particular execution:
- `inter_node_communication_enabled` will ensure nodes are allocated such that they can communicate with each other (e.g., send and receive network packets)

**Note:** Most often it is better to scale up the execution first, prior to scale out. Due to the default Batch core quota of just 20 cores, we are using 3 `STANDARD_NC6` nodes. In real production runs, we'd most likely scale up to multiple GPUs within a single node (parallel execution) such as `STANDARD_NC12` or `STANDARD_NC24` prior to scaling out to multiple NC nodes (parallel and distributed execution). We can further improve performance by opting to utilize the `STANDARD_NC24r` instances which are Infiniband/RDMA-capable with GPUs. To use this VM, we would also change the `IMAGE` to use the `intelmpi` versions of the Docker images along with `OpenLogic CentOS-HPC 7.3` as the `platform_image`.

In [ ]:
POOL_ID = 'gpupool-multi-instance'

pool = {
    "pool_specification": {
        "id": POOL_ID,
        "vm_configuration": {
            "platform_image": {
                "publisher": "Canonical",
                "offer": "UbuntuServer",
                "sku": "16.04-LTS"
            },
        },
        "vm_size": "STANDARD_NC6",
        "vm_count": {
            "dedicated": 3
        },
        "ssh": {
            "username": "docker"
        },
        "inter_node_communication_enabled": True
    }
}

In [ ]:
!mkdir config
write_json_to_file(config, os.path.join('config', 'config.json'))
write_json_to_file(pool, os.path.join('config', 'pool.json'))
print(json.dumps(config, indent=4, sort_keys=True))
print(json.dumps(pool, indent=4, sort_keys=True))

Create the pool, please be patient while the compute nodes are allocated.

In [ ]:
shipyard pool add -y

Ensure that all compute nodes are `idle` and ready to accept tasks:

In [ ]:
shipyard pool listnodes

<a id='section2'></a>

## Configure MPI Job and Submit

MPI jobs in Batch require execution as a multi-instance task. Essentially this allows multiple compute nodes to be used for a single task.

A few things to note in this jobs configuration:
- The `COMMAND` executes the `run_cntk.sh` script which is embedded into the Docker image. This helper script removes complexities needed in order to execute a distributed MPI CNTK job.
- `auto_complete` is being set to `True` which forces the job to move from `active` to `completed` state once all tasks complete. Note that once a job has moved to `completed` state, no new tasks can be added to it.
- `multi_instance` property is populated which enables multiple nodes, e.g., `num_instances` to participate in the execution of this task. The `coordination_command` is the command that is run on all nodes prior to the `command`. Here, we are simply executing the Docker image to run the SSH server for the MPI daemon (e.g., orted, hydra, etc.) to initialize all of the nodes prior to running the application command.

In [ ]:
JOB_ID = 'cntk-mpi-job'

# reduce the nubmer of epochs to 20 for purposes of this notebook
COMMAND = '/cntk/run_cntk.sh -s /cntk/Examples/Image/Classification/ResNet/Python/TrainResNet_CIFAR10_Distributed.py -- --network resnet20 -q 1 -a 0 --datadir /cntk/Examples/Image/DataSets/CIFAR-10 --outputdir $AZ_BATCH_TASK_WORKING_DIR/output'
jobs = {
    "job_specifications": [
        {
            "id": JOB_ID,
            "auto_complete": True,
            "tasks": [
                {
                    "image": IMAGE_NAME,
                    "command": COMMAND,
                    "gpu": True,
                    "multi_instance": {
                        "num_instances": "pool_current_dedicated",
                    }
                }
            ]
        }
    ]
}

In [ ]:
write_json_to_file(jobs, os.path.join('config', 'jobs.json'))
print(json.dumps(jobs, indent=4, sort_keys=True))

Submit the job and tail `stdout.txt`:

In [ ]:
shipyard jobs add --tail stdout.txt

Using the command below we can check the status of our jobs. Once all jobs have an exit code we can continue. You can also view the **heatmap** of this pool on [Azure Portal](https://portal.azure.com) to monitor the progress of this job on the compute nodes under your Batch account.

<a id='section3'></a>

## Delete Multi-instance Job

Deleting multi-instance jobs running as Docker containers requires a little more care. We will need to first ensure that the job has entered `completed` state. In the above `jobs` configuration, we set `auto_complete` to `True` enabling the Batch service to automatically complete the job when all tasks finish. This also allows automatic cleanup of the running Docker containers used for executing the MPI job.

Special logic is required to cleanup MPI jobs since the `coordination_command` that runs actually detaches an SSH server. The job auto completion logic Batch Shipyard injects ensures that these containers are killed.

In [ ]:
shipyard jobs listtasks

Once we are sure that the job is completed, then we issue the standard delete command:

In [ ]:
shipyard jobs del -y --termtasks --wait

In [ ]:
shipyard pool del -y --wait

[Next notebook: Advanced - Keras Single GPU Training with Tensorflow](08_Keras_Single_GPU_Training_With_Tensorflow.ipynb)